In [ ]:
import pandas as pd

# Load the dataset
# Replace with the actual path to your CSV file
df = pd.read_csv('/content/cleaned_dataset_combined (2).csv')

# Display the first few rows to verify
print("Original DataFrame:")
print(df.head())
print(df.info())

Original DataFrame:
   hate_speech  offensive_language  neither  class  \
0            0                   0        3      2   
1            0                   3        0      1   
2            0                   3        0      1   
3            0                   2        1      1   
4            0                   6        0      1   

                                               tweet  \
0  retwet as a woman you should not complain abou...   
1  retwet boy dats coldtyga dwn bad for cufin dat...   
2  retwet dawg retwet you ever fuck a bitch and s...   
3                        retwet she lok like a trany   
4  retwet the shit you hear about me might be tru...   

                                        tweet_tokens  
0  ['retwet', 'woman', 'complain', 'cleaning', 'h...  
1  ['retwet', 'boy', 'dat', 'coldtyga', 'dwn', 'b...  
2  ['retwet', 'dawg', 'retwet', 'ever', 'fuck', '...  
3                 ['retwet', 'lok', 'like', 'trany']  
4  ['retwet', 'shit', 'hear', 'might', 'tru

In [ ]:
import ast

# Convert string representation of list to actual list
df['tweet_tokens'] = df['tweet_tokens'].apply(ast.literal_eval)

# Display the first few rows to verify
print("\nDataFrame after converting tweet_tokens to lists:")
print(df.head())
print(df.info())


DataFrame after converting tweet_tokens to lists:
   hate_speech  offensive_language  neither  class  \
0            0                   0        3      2   
1            0                   3        0      1   
2            0                   3        0      1   
3            0                   2        1      1   
4            0                   6        0      1   

                                               tweet  \
0  retwet as a woman you should not complain abou...   
1  retwet boy dats coldtyga dwn bad for cufin dat...   
2  retwet dawg retwet you ever fuck a bitch and s...   
3                        retwet she lok like a trany   
4  retwet the shit you hear about me might be tru...   

                                        tweet_tokens  
0  [retwet, woman, complain, cleaning, house, man...  
1  [retwet, boy, dat, coldtyga, dwn, bad, cufin, ...  
2  [retwet, dawg, retwet, ever, fuck, bitch, star...  
3                         [retwet, lok, like, trany]  
4  [retwet, 

In [ ]:
from gensim.models import Word2Vec

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=df['tweet_tokens'], vector_size=100, window=5, min_count=1, workers=4)

# Get the vector size
vector_size = word2vec_model.wv.vector_size


In [ ]:
import numpy as np

def document_vector(doc):
    # Remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.wv.key_to_index]
    # If the document is empty, return a zero vector
    if not doc:
        return np.zeros(vector_size)
    # Calculate the mean vector
    return np.mean(word2vec_model.wv[doc], axis=0)

# Apply the function to transform tweet_tokens
X_vectors = df['tweet_tokens'].apply(document_vector).tolist()
X_vectors = np.array(X_vectors)


In [ ]:
# Combine Word2Vec features with other features
other_features = df[['hate_speech', 'offensive_language', 'neither']]
X_combined = np.hstack((other_features, X_vectors))

# Define the target variable
y = df['class']


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Create a pipeline with StandardScaler and RandomForestClassifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Standardize the features
    ('classifier', RandomForestClassifier())
])

# Train the model
pipeline.fit(X_train, y_train)

# Evaluate the model
accuracy = pipeline.score(X_test, y_test)
print(f'\nAccuracy: {accuracy}')



Accuracy: 0.9993947952390558
